In [1]:
import torch

In [2]:
def corr2d(X, K):
    kh, kw = K.shape
    xh, xw = X.shape
    
    Y = torch.zeros((xh-kh+1), (xw-kw+1))
    
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i+kh, j:j+kw] *K).sum()
    
    return Y

In [10]:
sum([2,3])

5

In [7]:
def corr2d_multi_in(X,K):
    return sum(corr2d(x,k) for x,k in zip(X,K))

In [34]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
print(X.shape, K.shape)
corr2d_multi_in(X, K)

torch.Size([2, 3, 3]) torch.Size([2, 2, 2])


tensor([[ 56.,  72.],
        [104., 120.]])

In [37]:
K = torch.normal(0,1,(1,1,1))
corr2d_multi_in(X,K).shape

torch.Size([3, 3])

In [12]:
[corr2d(x,k) for x,k in zip(X,K)]

[tensor([[19., 25.],
         [37., 43.]]),
 tensor([[37., 47.],
         [67., 77.]])]

In [24]:
def corr2d_multi_in_out(X,K):
    return torch.stack([corr2d_multi_in(X,k) for k in K],0)

In [25]:
K.shape

torch.Size([3, 2, 2, 2])

In [21]:
K = torch.stack((K, K+1, K+2),0)
K.shape

torch.Size([3, 2, 2, 2])

In [26]:
corr2d_multi_in_out(X,K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [27]:
def corr2d_multi_in_out_1x1(X,K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K,X)
    return Y.reshape((c_o, h, w))

In [28]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

In [30]:
Y1 = corr2d_multi_in_out_1x1(X,K)
Y2 = corr2d_multi_in_out(X,K)
print(Y1, Y2)

print(Y1==Y2)

tensor([[[ 4.7898,  0.0481, -2.9658],
         [-4.7554,  3.9008,  2.7700],
         [-5.2765,  0.6535,  4.6643]],

        [[-0.1877,  0.9995, -0.3257],
         [-2.4412,  0.0975,  2.5876],
         [-2.4499,  0.2897, -2.6085]]]) tensor([[[ 4.7898,  0.0481, -2.9658],
         [-4.7554,  3.9008,  2.7700],
         [-5.2765,  0.6535,  4.6643]],

        [[-0.1877,  0.9995, -0.3257],
         [-2.4412,  0.0975,  2.5876],
         [-2.4499,  0.2897, -2.6085]]])
tensor([[[True, True, True],
         [True, True, True],
         [True, True, True]],

        [[True, True, True],
         [True, True, True],
         [True, True, True]]])


### Exercises

1. Assume that we have two convolution kernels of size $k_1$ and $k_2$, respectively (with no nonlinearity in between).
    1. Prove that the result of the operation can be expressed by a single convolution.
    * not sure
    1. What is the dimensionality of the equivalent single convolution?
    * dont know
    1. Is the converse true?
    * dont know
    
1. Assume an input of shape $c_i\times h\times w$ and a convolution kernel of shape $c_o\times c_i\times k_h\times k_w$, padding of $(p_h, p_w)$, and stride of $(s_h, s_w)$.
    1. What is the computational cost (multiplications and additions) for the forward propagation?
    1. What is the memory footprint?
    1. What is the memory footprint for the backward computation?
    1. What is the computational cost for the backpropagation?
1. By what factor does the number of calculations increase if we double the number of input channels $c_i$ and the number of output channels $c_o$? What happens if we double the padding?
1. If the height and width of a convolution kernel is $k_h=k_w=1$, what is the computational complexity of the forward propagation?
1. Are the variables `Y1` and `Y2` in the last example of this section exactly the same? Why?
1. How would you implement convolutions using matrix multiplication when the convolution window is not $1\times 1$?


In [38]:
# 1

k1 = torch.normal(0,1,(2,3,2,2))
k2 = torch.normal(0,1,(2,3,2,2))

X = torch.normal(0,1,(3,3,3))

In [39]:
Y = corr2d_multi_in_out(X,k1)
print(Y)

corr2d_multi_in_out(Y, k2)

tensor([[[-2.3365, -4.8509],
         [-1.5722,  6.2972]],

        [[-5.6773,  0.4476],
         [ 1.5267,  1.3301]]])


tensor([[[ -8.0220]],

        [[-16.6490]]])

In [40]:
k1 = k1 + k2

In [41]:
corr2d_multi_in_out(X, k1)

tensor([[[ -1.8779,  -5.7017],
         [ -6.9105,   4.1614]],

        [[-10.3369,   0.2607],
         [ -1.6612,   0.5963]]])